In [148]:
import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp import *
import os
import csv
import pandas as pd
pdb_path = '/home/tongwade780/pdb_website/onebyone_cluster_code/muscle_alignment_pdb'
pdbname_file = open ('/home/tongwade780/pdb_website/all_ca_number_all_code/onechain_100clust.csv').readlines ()
next_file = []
for line in pdbname_file[0:]:
    next_file.append(line.split(','))
#for pdb_chain_name in next_file[0]:
    new_pdb_chain_name = filter(None, pdb_chain_name)
    pdbname = new_pdb_chain_name[0:4].lower()
    protein_chain = new_pdb_chain_name[5]
    pdb_file = '{0}{1}'.format(pdbname,'.pdb')
    pdbPATH = os.path.join(pdb_path,pdb_file)

    #sasa table
    sasa_result = getSASA_final(pdbPATH)
    #dssp table
    final_dssp_table = getDSSP_final(pdbPATH)
    #pka table
    pka_final_result = getcharge_pKa(pdbPATH)

NameError: name 'pdb_chain_name' is not defined

In [3]:
import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp_new import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 
#pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdbname_file = open ('/home/tongwade780/pdb_website/all_ca_number_all_code/onechain_100clust.csv').readlines ()
next_file = []
for line in pdbname_file[0:]:
    next_file.append(line.rstrip('\r\n' + '').split(','))
    #next_file.append(line.split(','))
delet_file = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/cluster_delet_pdb_index/cluster_{0}_delet_pdb_index.mat'.format(1),'r') 
delet_file['delet_pdb_index']

delet_index = delet_file['delet_pdb_index'].value.T
pdb_chain_name = next_file[0]

new_pdb_chain_name =  np.asarray(pdb_chain_name)

new_pdb_chain_name = np.delete(new_pdb_chain_name,delet_index[0]-1,None)

new_pdb_chain_name = filter(None, new_pdb_chain_name)


#pdbname= new_pdb_chain_name[0][0:4].lower()
pdbname = '3eqi'
pdb_file = '{0}{1}'.format(pdbname,'.pdb')
pdbPATH = os.path.join(pdb_path,pdb_file)

#sasa table
sasa_result = getSASA_final(pdbPATH)
#dssp table
final_dssp_table = getDSSP_final(pdbPATH)
#pka table
pka_final_result = getcharge_pKa(pdbPATH)

#load matlab data with dataframe 

f = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/feature_matrix/{0}_individual_feature.mat'.format(pdbname),'r') 

datas = f['individual_feature'].value.T
matlab_data = pd.DataFrame(datas,columns=['resID','eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2',
                                          'Rank_2','GNM_co','co_rank','entropy','rmsf_overall','pca_feature',
                                          'eig_first','eig_sec','shannon_entropy','shannon_entropy20',
                                          'eig5_eig1','atomic_1','atomic_3','atomic_5','ssbond_matrix',
                                          'Polarity','charge','stand_Dcom','side_chain_length','h_bond_group',
                                          'ca_len','hole_ss_bond','contact_per_res','gyradius',
                                          'phobic_percent','intrinsicDisorder','RMSF','RMSF_0f_bfactor','shifted_RMSF'])

#change resID value type to int
matlab_data['resID'] = matlab_data['resID'].astype('int')

sasa_result.columns = ['resID','sasa']
sasa_result['resID'] = sasa_result['resID'].astype('int')
pka_final_result.columns = ['resID','pka_charge']
pka_final_result['resID'] = pka_final_result['resID'].astype('int')
final_dssp_table.columns = ['resID','dssp_result','loop_percent']
# final_dssp_table['resID'] = sasa_result['resID'].values
final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)
final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
final_dssp_table['resID'] = final_dssp_table['resID'].astype('int')
#merge matlab feature and python feature
merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')

merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')

merged_data['pka_charge']=merged_data['pka_charge'].fillna(0)

merged_data = merged_data.merge(final_dssp_table,how = 'left',on = 'resID')
#add pdb id
res_len = len(final_dssp_table['dssp_result'].values)

merged_data['PDB_ID'] = pd.Series(np.repeat(pdbname, res_len))
#change feature order
change_order = ['PDB_ID','resID','atomic_1','atomic_3','atomic_5','ssbond_matrix','Polarity',
                'eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2','Rank_2',
                'GNM_co','co_rank','charge','stand_Dcom','side_chain_length','pka_charge',
                'dssp_result','sasa','entropy','ca_len','hole_ss_bond','rmsf_overall',
                'contact_per_res','gyradius','pca_feature','eig_first','eig_sec',
                'shannon_entropy','shannon_entropy20','eig5_eig1','phobic_percent',
                'loop_percent','h_bond_group','intrinsicDisorder','RMSF','RMSF_0f_bfactor',
                'shifted_RMSF']

merged_data = merged_data[change_order]

propka3.1                                                                                    2017-08-10
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M

In [49]:
merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')
merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')
merged_data

,resID,eig_vv_1,eig_vv_2,vector_1,Rank_1,vector_2,Rank_2,GNM_co,co_rank,entropy,...,hole_ss_bond,contact_per_res,gyradius,phobic_percent,intrinsicDisorder,RMSF,RMSF_0f_bfactor,shifted_RMSF,sasa,pka_charge
0,2,0.058135,0.060651,0.013143,0.068182,0.028901,0.446970,0.224559,0.643939,60.505264,...,4.0,23.030303,25.084123,57.575758,0.467471,0.372733,0.822183,0.846043,42.6,1.0
1,3,0.023851,0.094244,0.005392,0.015152,0.044907,0.560606,0.225427,0.659091,60.505264,...,4.0,23.030303,25.084123,57.575758,0.431325,0.370633,0.805678,0.843943,67.6,NaN
2,4,0.134263,0.136612,0.030354,0.174242,0.065096,0.651515,0.180387,0.340909,60.505264,...,4.0,23.030303,25.084123,57.575758,0.424210,0.318186,0.799133,0.791496,6.1,NaN
3,5,0.212151,0.171844,0.047962,0.242424,0.081884,0.696970,0.208582,0.522727,60.505264,...,4.0,23.030303,25.084123,57.575758,0.407962,0.347708,0.830154,0.821018,45.4,NaN
4,6,0.297288,0.188774,0.067210,0.371212,0.089951,0.750000,0.188866,0.409091,60.505264,...,4.0,23.030303,25.084123,57.575758,0.392005,0.381605,0.825573,0.854915,35.3,1.0
5,7,0.348979,0.240810,0.078896,0.553030,0.114747,0.825758,0.203410,0.484848,60.505264,...,4.0,23.030303,25.084123,57.575758,0.374178,0.350919,0.831701,0.824229,33.8,0.0
6,8,0.299468,0.203477,0.067703,0.378788,0.096957,0.803030,0.201382,0.477273,60.505264,...,4.0,23.030303,25.084123,57.575758,0.365392,0.311801,0.800319,0.785111,49.0,-1.0
7,9,0.301835,0.190316,0.068238,0.401515,0.090686,0.757576,0.167102,0.250000,60.505264,...,4.0,23.030303,25.084123,57.575758,0.374121,0.315128,0.756184,0.788439,0.0,NaN
8,10,0.341265,0.192193,0.077152,0.507576,0.091580,0.765152,0.155155,0.181818,60.505264,...,4.0,23.030303,25.084123,57.575758,0.385365,0.314586,0.768920,0.787896,0.0,NaN
9,11,0.358269,0.225986,0.080996,0.583333,0.107683,0.818182,0.195738,0.439394,60.505264,...,4.0,23.030303,25.084123,57.575758,0.393801,0.221675,0.808409,0.694985,46.5,NaN


0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
5      0.5
6      0.5
7      0.5
8      0.5
9      0.5
10     0.5
11     0.5
12     0.5
13     0.5
14     0.5
15     0.5
16     0.5
17     0.5
18     0.5
19     0.5
20     1.0
21     1.0
22     1.0
23     1.0
24     1.0
25     1.0
26     0.5
27     0.5
28     0.5
29     0.0
      ... 
285    0.5
286    0.5
287    0.5
288    0.5
289    1.0
290    1.0
291    0.5
292    0.5
293    0.5
294    0.5
295    0.5
296    0.5
297    0.5
298    0.5
299    1.0
300    1.0
301    1.0
302    1.0
303    0.5
304    0.5
305    0.5
306    0.5
307    0.5
308    0.5
309    0.5
310    0.5
311    0.5
312    1.0
313    1.0
314    NaN
Name: dssp_result, dtype: float64

In [2]:
merged_data

NameError: name 'merged_data' is not defined

In [32]:
f2['individual_feature'] = f['individual_feature'][...].astype('float64')

NameError: name 'f2' is not defined

In [9]:
sasa_result 

,resID,sasa
0,1,57.8
1,2,24.7
2,3,95.4
3,4,42.6
4,5,65.9
5,6,92.0
6,7,49.2
7,8,21.3
8,9,53.9
9,10,58.7


In [18]:
datas

array([[  2.00000000e+00,   7.01959162e-02,   6.47566279e-02, ...,
          3.72435443e-01,   8.22172198e-01,   8.45961179e-01],
       [  3.00000000e+00,   2.12461982e-02,   9.70057243e-02, ...,
          3.70645273e-01,   8.05677592e-01,   8.44171009e-01],
       [  4.00000000e+00,   1.38433657e-01,   1.41375265e-01, ...,
          3.18191138e-01,   7.99133351e-01,   7.91716874e-01],
       ..., 
       [  1.28000000e+02,   5.15110881e-01,   3.48292062e-01, ...,
          3.12849667e-01,   9.47128805e-01,   7.86375403e-01],
       [  1.29000000e+02,   5.76848983e-01,   4.49867055e-01, ...,
          5.73023581e-01,   1.06960763e+00,   1.04654932e+00],
       [  1.30000000e+02,   5.35205145e-01,   4.11423518e-01, ...,
          8.28334486e-01,   1.16485078e+00,   1.30186022e+00]])

# ---------------------------------------------------------------------

In [ ]:
import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 
pdb_path = '/home/tongwade780/pdb_website/onebyone_cluster_code/muscle_alignment_pdb'
pdbname_file = open ('/home/tongwade780/pdb_website/all_ca_number_all_code/onechain_100clust.csv').readlines ()
next_file = []
for line in pdbname_file[0:]:
    next_file.append(line.rstrip('\r\n' + '').split(','))
    #next_file.append(line.split(','))
pdb_chain_name = next_file[1]

new_pdb_chain_name =  np.asarray(pdb_chain_name)

new_pdb_chain_name = np.delete(new_pdb_chain_name,delet_index[0]-1,None)

new_pdb_chain_name = filter(None, new_pdb_chain_name)

In [ ]:
res_len = len(final_dssp_table['dssp_result'].values)

loop_num = np.count_nonzero(final_dssp_table['dssp_result'].values == 1)

loop_percent = float (loop_num)/float (res_len) * float (100)
final_dssp_table['loop_percent'] = pd.Series(np.repeat(loop_percent, res_len))

In [3]:
delet_file = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/cluster_delet_pdb_index/cluster_{0}_delet_pdb_index.mat'.format(1),'r') 
delet_file['delet_pdb_index']

delet_index = delet_file['delet_pdb_index'].value.T

In [2]:
import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp_new import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 
#pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdbname_file = open ('/home/tongwade780/pdb_website/all_ca_number_all_code/onechain_100clust.csv').readlines ()
next_file = []
for line in pdbname_file[0:]:
    next_file.append(line.rstrip('\r\n' + '').split(','))
    #next_file.append(line.split(','))
delet_file = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/cluster_delet_pdb_index/cluster_{0}_delet_pdb_index.mat'.format(1701),'r') 
delet_file['delet_pdb_index']

delet_index = delet_file['delet_pdb_index'].value.T
pdb_chain_name = next_file[0]

new_pdb_chain_name =  np.asarray(pdb_chain_name)

new_pdb_chain_name = np.delete(new_pdb_chain_name,delet_index[0]-1,None)

new_pdb_chain_name = filter(None, new_pdb_chain_name)


#pdbname= new_pdb_chain_name[0][0:4].lower()
pdbname = '2cl3'
pdb_file = '{0}{1}'.format(pdbname,'.pdb')
pdbPATH = os.path.join(pdb_path,pdb_file)

#sasa table
sasa_result = getSASA_final(pdbPATH)
#dssp table
final_dssp_table = getDSSP_final(pdbPATH)
try:
    #pka table
    pka_final_result = getcharge_pKa(pdbPATH)
except:
    problem_pdb.append((pdbname,'getcharge_pKa broke'))
#     continue
    #load matlab data with dataframe 

f = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/feature_matrix_2/{0}_individual_feature.mat'.format(pdbname),'r') 

datas = f['individual_feature'].value.T
matlab_data = pd.DataFrame(datas,index=datas[:,0],columns=['resID','eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2',
                                          'Rank_2','GNM_co','co_rank','entropy','rmsf_overall','pca_feature',
                                          'eig_first','eig_sec','shannon_entropy','shannon_entropy20',
                                          'eig5_eig1','atomic_1','atomic_3','atomic_5','ssbond_matrix',
                                          'Polarity','charge','stand_Dcom','side_chain_length',
                                          'ca_len','hole_ss_bond','contact_per_res','gyradius',
                                          'phobic_percent','RMSF','RMSF_0f_bfactor','shifted_RMSF',
                                                          'h_bond_group','intrinsicDisorder','pca_featrue_2'])

#change resID value type to int
matlab_data['resID'] = matlab_data['resID'].astype('int')

sasa_result.columns = ['resID','sasa']

sasa_result['resID'] = sasa_result['resID'].astype('int')

pka_final_result.columns = ['resID','pka_charge']
pka_final_result['resID'] = pka_final_result['resID'].astype('int')
final_dssp_table.columns = ['resID','dssp_result','loop_percent','sheet_percent','helix_percent']
final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)
final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
#   try:
#       final_dssp_table['resID'] = sasa_result['resID'].values
#   except:
#       problem_pdb.append((pdbname,'get_dssp_merge broke'))
#       continue    
final_dssp_table['resID'] = final_dssp_table['resID'].astype('int')

#merge matlab feature and python feature
merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')

merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')

merged_data['pka_charge']=merged_data['pka_charge'].fillna(0)

merged_data = merged_data.merge(final_dssp_table,how = 'left',on = 'resID')
#add pdb id
merged_data['PDB_ID'] = pdbname
#change feature order
change_order = ['PDB_ID','resID','atomic_1','atomic_3','atomic_5','ssbond_matrix','Polarity',
                'eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2','Rank_2',
                'GNM_co','co_rank','charge','stand_Dcom','side_chain_length','pka_charge',
                'dssp_result','sasa','entropy','ca_len','hole_ss_bond','rmsf_overall',
                'contact_per_res','gyradius','pca_feature','eig_first','eig_sec',
                'shannon_entropy','shannon_entropy20','eig5_eig1','phobic_percent',
                'loop_percent','h_bond_group','intrinsicDisorder','pca_featrue_2','RMSF','RMSF_0f_bfactor',
                'shifted_RMSF']

merged_data = merged_data[change_order]


propka3.1                                                                                    2017-11-10
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M

In [3]:
merged_data

,PDB_ID,resID,atomic_1,atomic_3,atomic_5,ssbond_matrix,Polarity,eig_vv_1,eig_vv_2,vector_1,...,shannon_entropy20,eig5_eig1,phobic_percent,loop_percent,h_bond_group,intrinsicDisorder,pca_featrue_2,RMSF,RMSF_0f_bfactor,shifted_RMSF
0,2cl3,2,2.0,0.000000,0.000000,0.0,9.0,4.385744,1.194685,0.344442,...,0.906049,0.727051,50.0,38.916256,0.0,0.555400,0.781,0.546071,1.263962,1.449989
1,2cl3,3,1.0,0.000000,0.000000,0.0,11.6,4.273626,1.135543,0.335636,...,0.906049,0.727051,50.0,38.916256,0.0,0.542016,0.781,0.268175,1.172380,1.172093
2,2cl3,4,3.0,0.000000,0.000000,0.0,11.3,4.153421,1.073485,0.326196,...,0.906049,0.727051,50.0,38.916256,0.0,0.498353,0.781,0.314879,1.166813,1.218798
3,2cl3,5,3.0,0.000000,0.000000,0.0,6.2,3.946076,0.967046,0.309912,...,0.906049,0.727051,50.0,38.916256,0.0,0.512048,0.781,0.291083,1.177611,1.195002
4,2cl3,6,3.0,0.000000,0.000000,0.0,5.2,3.682042,0.834622,0.289175,...,0.906049,0.727051,50.0,38.916256,0.0,0.550498,0.781,0.251874,1.107654,1.155792
5,2cl3,7,3.0,0.000000,0.000000,0.0,10.5,3.365302,0.678555,0.264300,...,0.906049,0.727051,50.0,38.916256,0.0,0.575846,0.781,0.279309,1.139057,1.183228
6,2cl3,8,4.0,0.000000,1.800000,0.0,10.5,2.974193,0.493412,0.233583,...,0.906049,0.727051,50.0,38.916256,0.0,0.591281,0.781,0.233618,1.145710,1.137537
7,2cl3,9,5.0,3.000000,1.800000,0.0,8.6,2.594599,0.316711,0.203771,...,0.906049,0.727051,50.0,38.916256,0.0,0.587229,0.781,0.163817,1.133548,1.067735
8,2cl3,10,13.0,5.000000,1.200000,0.0,11.3,2.021246,0.069279,0.158742,...,0.906049,0.727051,50.0,38.916256,0.0,0.574881,0.781,0.260547,1.163514,1.164465
9,2cl3,11,15.0,5.000000,0.000000,0.0,8.0,1.813883,0.022212,0.142456,...,0.906049,0.727051,50.0,38.916256,2.0,0.580463,0.781,0.193032,1.119086,1.096950
